##  Relative Engine

The `AbsoluteEnhancedRandomEngine.analyse` method
is not very effective as it always recalculates the whole board. Thus, it does up to 16 calculations. 
With one move, the evaluation can change in the following ways: 
* There is a checkmate
* There is a draw
* Capture of a piece
* Promotion of a piece
* Capture and promotion of a piece
* Nothing changes

With this idea in mind, the next engine implements an incremental heuristic.

In [ ]:
class RelativeEnhancedRandomEngine(AbsoluteEnhancedRandomEngine):
    """Chess engine using the change of value of the chessboard to evalute moves"""

The new method `_relative_heuristic` takes a board and a move as a parameter and returns the change of the score by playing that move from the perspective of the player to move.  
If a figure is captured, 
the overall score of the board 
will increase 
by its value.
If a pawn is promoted, 
the overall score of the board 
will be increased 
by the value of the promoted piece 
minus the value of the pawn.
The score is returned as is if it is White's turn, otherwise it is returned negated.

In [ ]:
def _relative_heuristic(self, board: chess.Board, move: chess.Move) -> int:
    value = 0
    if capturedPiece := board.piece_type_at(move.to_square):
        value += self.pieceValue(capturedPiece)
    if promotionPiece := move.promotion:
        value += self.pieceValue(promotionPiece) - self.VALUE_PAWN

    return value if board.turn is chess.WHITE else -value


RelativeEnhancedRandomEngine._relative_heuristic = _relative_heuristic

The `analyse` method is similar to the one defined for the `AbsoluteEnhancedRandomEngine` class. 
The only difference is 
that the initial score of the board 
is calculated first by using the `_absolute_heuristic` method
and the `_evaluate_move` method taking an additional parameter `currentScore`. 
In the loop, the value of each move is the sum of the inital score and the value calculated by `_relative_heuristic` of the move.

In [ ]:
def _evaluate_move(
    self: Engine, board: chess.Board, move: chess.Move, currentScore: int
) -> int:
    """Evaluate a single move using _relative_heuristic"""
    whitesTurn = board.turn is chess.WHITE
    score = currentScore + self._relative_heuristic(board, move)
    board.push(move)
    if board.is_checkmate():
        score = self.VALUE_CHECKMATE if whitesTurn else -self.VALUE_CHECKMATE
    elif board.is_draw():
        score = self.VALUE_DRAW
    board.pop()
    return score


RelativeEnhancedRandomEngine._evaluate_move = _evaluate_move


def analyse(self, board: chess.Board) -> list[ScoredMove]:
    """Analyse method using _relative_heuristic"""
    currentScore = self._absolute_heuristic(board)
    whitesTurn = board.turn is chess.WHITE
    nextMoves = []
    for move in board.legal_moves:
        score = self._evaluate_move(board, move, currentScore)

        nextMoves.append(ScoredMove(score, move))

    random.shuffle(nextMoves)
    nextMoves.sort(reverse=whitesTurn)
    return nextMoves


RelativeEnhancedRandomEngine.analyse = analyse

As both engines should play the same move, the outcome of the game against the `OpeningRandomEngine` is the same provided the seed is the same.

In [ ]:
random.seed(42)
board_relative = chess.Board()
playGame(
    board_relative,
    RelativeEnhancedRandomEngine(),
    AbsoluteEnhancedRandomEngine(),
    displayBoard=False
)
IPython.display.display(board)
print(board_relative.outcome())

To verfify that both engines play exactly the same moves, the following test compares the move stacks of boths games.

In [ ]:
assert board_relative.move_stack == board_absolute.move_stack, f"There were different moves"

Performance measurements

TO DO: 
- More details (statistics)

In [ ]:
random.seed(42)
absoluteEngine = AbsoluteEnhancedRandomEngine()
relativeEngine = RelativeEnhancedRandomEngine()

board1 = chess.Board(SHIROV_SACRIFICE)
board2 = chess.Board(VLADIMIROV_THUNDERBOLT)
board3 = chess.Board(PROMOTION_POSITION)

print("Absolute Engine")
random.seed(42)
%timeit -r 2 -n 5 absoluteEngine.analyse(board1)
%timeit -r 2 -n 5 absoluteEngine.analyse(board2)
%timeit -r 2 -n 5 absoluteEngine.analyse(board3)

print("Relative Engine")
random.seed(42)
%timeit -r 2 -n 5 relativeEngine.analyse(board1)
%timeit -r 2 -n 5 relativeEngine.analyse(board2)
%timeit -r 2 -n 5 relativeEngine.analyse(board3)